In [33]:
GIT_ROOT_LINES = !git rev-parse --show-toplevel
WORK_DIR = GIT_ROOT_LINES[0]
%cd $WORK_DIR
%load_ext autoreload
%autoreload 2

# for pyrender to work in a headless environment (to render graphs)
# you may remove the following two lines if you run this in a machine with GUI
%env PYOPENGL_PLATFORM=egl
%env EGL_DEVICE_ID=0

env: PYOPENGL_PLATFORM=egl
env: EGL_DEVICE_ID=0


In [34]:
from motionblender.train import *
from motionblender.init_utils import *
from motionblender.lib.misc import *
import mediapy
from motionblender.app.main import MeshRenderBackend
import pyrender

In [35]:
def get_full_splat_at_t(gses, motions, t:int, w2c=None):
    full_splats = {}
    for g in gses.keys():
        if g not in motions:
            means, quats = gses[g].means, gses[g].get_quats()
        else:
            means, quats = motions[g].transform_splats_to_t(gses[g].means, t, cano_quats_wxyz=gses[g].get_quats())
        full_splats.setdefault('means', []).append(means)
        full_splats.setdefault('quats', []).append(quats)
        full_splats.setdefault("scales", []).append(gses[g].get_scales())
        full_splats.setdefault("opacities", []).append(gses[g].get_opacities())

        if w2c is not None:
            assert w2c.dim() == 3
            cam_center = torch.inverse(w2c)[:, :3, 3].to(means.device)
        else:
            cam_center = None
        full_splats.setdefault("colors", []).append(gses[g].get_colors(means, cam_center))
    full_splats = {k:torch.cat(v) for k, v in full_splats.items()}
    return full_splats

In [37]:
IMG_WH = [720, 960]
cfg =  tyro.cli(TrainConfig, args=["--work-dir", "outputs/iphone/handwavy", "data:iphone", "--data.data-dir", "./datasets/iphone/handwavy", "--data.cache-version", "jan18", '--data.load-from-cache'])

In [38]:
mesh_renderer = MeshRenderBackend()
mesh_renderer.pyrender = pyrender.OffscreenRenderer(viewport_width=IMG_WH[0], viewport_height=IMG_WH[1])

trigger(trigger__17)
trigger(trigger__18)
trigger(P_0x2b770f144370_2Camera)
trigger(P_0x2b770f144370_2AnimateStart)
trigger(P_0x2b770f144370_2AnimateStop)
js_key = class
js_key = style
js_key = fluid
js_key = class
before: class = { 'rounded-circle': !P_0x2b770f144370_2_show_ui }
(prefix=None) token {
has({ => {) = False
(prefix=None) token  
has(  =>  ) = False
(prefix=None) token '
has(' => ') = False
(prefix=None) token rounded
has(rounded => rounded) = False
(prefix=None) token -
has(- => -) = False
(prefix=None) token circle
has(circle => circle) = False
(prefix=None) token '
has(' => ') = False
(prefix=None) token :
has(: => :) = False
(prefix=None) token  
has(  =>  ) = False
(prefix=None) token !
has(! => !) = False
(prefix=None) token P_0x2b770f144370_2_show_ui
has(P_0x2b770f144370_2_show_ui => P_0x2b770f144370_2_show_ui) = True
(prefix=None) translated P_0x2b770f144370_2_show_ui
(prefix=None) token  
has(  =>  ) = False
(prefix=None) token }
has(} => }) = False
 => { 'rounded

2025-05-05 13:30:37.032 | INFO     | motionblender.lib.pv:url:495 - pyvista viewer url: http://localhost:46326/index.html?ui=P_0x2b770f144370_2&reconnect=auto


In [5]:
train_dataset, video_dataset = get_train_val_datasets(cfg.data, load_val=False)[:2]

2025-05-05 13:20:21.584 | INFO     | flow3d.data.iphone_dataset:__init__:104 - self.time_ids.min()=tensor(0) self.time_ids.max()=tensor(302)
2025-05-05 13:20:21.586 | INFO     | flow3d.data.iphone_dataset:__init__:108 - self.num_frames=303


max_depth_value tensor(8.1208)


2025-05-05 13:20:58.776 | INFO     | flow3d.data.iphone_dataset:__init__:275 - len(self.query_tracks_2d)=303 self.query_tracks_2d[0].shape=torch.Size([5216, 4])


loading cached scene norm dict...


2025-05-05 13:21:02.502 | WARNING  | motionblender.lib.dataset:__init__:103 - MotionBlenderIPhoneDataset cache tag -> jan18


In [11]:
gses, motions, _, _ = load_cpkl('outputs/ckpt/iphone/handwavy/ckpt.cpkl')

In [51]:
frames = []
for t, sample in enumerate(tqdm(video_dataset)):
    splat_dict = get_full_splat_at_t(gses, motions, t)
    output = render(splat_dict, sample['w2cs'].cuda(), sample['Ks'].cuda(), img_wh = [720, 960])
    rgb = output[0][0, :, :, :3].clamp_(0, 1).cpu()

    joints, links = motions['person-1']._joints_tensor_cache[t], motions['person-1'].links

    # render graph can be slow
    mesh_renderer.update_scene(joints, links,
                ['red']* len(joints), 
                ['blue']*len(links), 
                sample['Ks'],
                intensity=20.,
                link_radius=0.06
            )
    mesh_renderer.update_camera(sample['w2cs'].clone())
    graph_color, graph_depth, graph_mask = mesh_renderer.render()
    graph_mask = torch.from_numpy(graph_mask[..., None]).float()

    link_opacity = 0.5
    graph_rgb_img = rgb * (1 - graph_mask) + graph_mask * ((torch.from_numpy(graph_color.copy()).float() / 255) * link_opacity + rgb * (1 - link_opacity))
    frames.append((graph_rgb_img * 255).byte().cpu().numpy())

frames = np.stack(frames, axis=0)

  0%|          | 0/303 [00:00<?, ?it/s]

In [52]:
mediapy.show_video(frames, fps=15) # if this step fails with unknown option or something, try to reinstall your `ffmpeg` with `conda install ffmpeg -c conda-forge` maybe

This browser does not support the video tag.
